In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import Control, Coupling, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q0",
        dimension=3,
        frequency=7.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q1",
        dimension=3,
        frequency=8.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q0", "Q1"), strength=0.01),
]

system = QuantumSystem(
    objects=transmons,
    couplings=couplings,
)

In [ ]:
simulator = QuantumSimulator(system)

In [ ]:
control_qubit = transmons[0]
target_qubit = transmons[1]

In [ ]:
dt = 0.1  # ns
qp.set_sampling_period(dt)
Control.SAMPLING_PERIOD = dt

In [ ]:
pi_pulse = qp.Rect(duration=20, amplitude=2 * np.pi * 0.025)
cr_pulse = qp.FlatTop(duration=300, amplitude=2 * np.pi * 0.2, tau=30)

with qp.PulseSchedule(
    [
        "Q0-Q1",
        "Q0",
    ]
) as ps:
    ps.add("Q0-Q1", cr_pulse)
    # ps.barrier()
    # ps.add("Q0", pi_pulse)
    # ps.barrier()
    # ps.add("Q0-Q1", cr_pulse)
    # ps.barrier()
    # ps.add("Q0", pi_pulse)

ps.plot()

In [ ]:
seq = ps.get_sampled_sequences()

In [ ]:
control_cr = Control(
    target=control_qubit.label,
    frequency=target_qubit.frequency,
    waveform=seq["Q0-Q1"],
)
control_cr.plot()

In [ ]:
control_pi = Control(
    target=control_qubit.label,
    frequency=control_qubit.frequency,
    waveform=seq["Q0"],
)
control_pi.plot()

In [ ]:
result = simulator.mesolve(
    controls=[
        control_cr,
        # control_pi,
    ],
    initial_state=system.state(
        {
            "Q0": "0",
            "Q1": "0",
        },
    ),
)

result.plot_state_vectors("Q0")
result.plot_state_vectors("Q1")

result.display_bloch_sphere("Q0")
result.display_bloch_sphere("Q1")

result.plot_population_dynamics("Q0")
result.plot_population_dynamics("Q1")
result.plot_population_dynamics()

In [ ]:
result = simulator.mesolve(
    controls=[control_cr, control_pi],
    initial_state=system.state(
        {
            "Q0": "1",
            "Q1": "0",
        },
    ),
)

result.plot_state_vectors("Q0")
result.plot_state_vectors("Q1")

result.display_bloch_sphere("Q0")
result.display_bloch_sphere("Q1")

result.plot_population_dynamics("Q0")
result.plot_population_dynamics("Q1")
result.plot_population_dynamics()